# Most basic time decomposition simulation
We assume a flare with fixed physical parameters and two basis light curves.

The basis light curves are fractional browninan motion with $H = 0.5$ and $H = 0.955$.

The instrument response matrix is ignored for now.

The background is randomly distributed around a mean value,
with $\sigma = 4\mu$,
such that the nonthermal component appears washed out above 50 keV.

The physical parameters are as follows

**Thermal, isothermal**
- $\text{T} = 20$ MK
- $\text{EM} = 10^{49} \text{ cm}^{-3}$

**Nonthermal, thick target**
- $\varphi_e = 2\times 10^{35}$ electron/second
- $\gamma = 4$
- $\text{E}_c = 16$ keV

In [ ]:
import os

from astropy import units as u
from astropy import visualization as viz
from matplotlib import pyplot as plt
import numpy as np

from yaff import common_models as cm
from yaff.fitting import Parameter

from tedec import fractional_brownian_motion as fbm
from tedec import decomp

%matplotlib qt
plt.style.use(os.getenv('MPL_INTERACTIVE_STYLE'))

In [ ]:
thermal_physical_params = {
    'temperature': Parameter(20 << u.MK, True),
    'emission_measure': Parameter(1 << (1e49 * u.cm**-3), True)
}

nonthermal_physical_params = {
    'electron_flux': Parameter(2 << (1e35 * u.electron / u.s), True),
    'spectral_index': Parameter(4 << u.one, True),
    'cutoff_energy': Parameter(16 << u.keV, True)
}

energies = np.geomspace(3, 100, num=40) << u.keV

all_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': (thermal_physical_params | nonthermal_physical_params)
}

thermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': thermal_physical_params
}

nonthermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': nonthermal_physical_params
}

In [ ]:
def model(params: cm.ArgsT):
    return cm.thermal(params) + cm.thick_target(params)

In [ ]:
fig, ax = plt.subplots()

de = energies[1:] - energies[:-1]
integration_window = 30 << u.s
area = 10 << u.cm**2

with viz.quantity_support():
    flux_unit = (u.ph / u.cm**2 / u.s / u.keV)

    thermal_flux = cm.thermal(thermal_args) << flux_unit
    thermal_photons = (thermal_flux * de * integration_window * area).to(u.ph)
    del thermal_flux
    ax.stairs(thermal_photons, energies, label='thermal', color='red')

    nonthermal_flux = cm.thick_target(nonthermal_args) << flux_unit
    nonthermal_photons = (nonthermal_flux * de * integration_window * area).to(u.ph)
    del nonthermal_flux
    ax.stairs(nonthermal_photons, energies, label='nonthermal', color='blue')

    total_flux = model(all_args) << flux_unit
    total_photons = (total_flux * de * integration_window * area).to(u.ph)
    del total_flux
    ax.stairs(total_photons, energies, label='total', color='black', linestyle='dashed')

ax.set(xscale='log', yscale='log', ylim=(100, None))
ax.legend()
plt.show()

In [ ]:
seed = 132457
np.random.seed(seed)
time_bin = 0.1
integration = 30
steps = int(integration / time_bin)
thermal_basis = fbm.make_timeseries(num=steps, hurst=0.955)
nonthermal_basis = fbm.make_timeseries(num=steps, hurst=0.5)

In [ ]:
def rebin_clumps(histogram, clump_size):
    ret = np.zeros(histogram.size // clump_size)
    for i in range(0, histogram.size, clump_size):
        ret[i // clump_size] = histogram[i:i+clump_size].sum()
    return ret

# Bin down the time granularity
real_dt = 0.5
bin_down_factor = int(real_dt / time_bin)
thermal_basis = rebin_clumps(thermal_basis, bin_down_factor)
nonthermal_basis = rebin_clumps(nonthermal_basis, bin_down_factor)

In [ ]:
fig, ax = plt.subplots()
axx = ax.twinx()
t = np.arange(thermal_basis.size + 1)
ax.stairs(thermal_basis, t, color='red', label='thermal')
ax.legend(loc='lower right')
ax.set(ylabel='thermal magnitude')

axx.stairs(nonthermal_basis, t, color='black', label='nonthermal')
axx.legend(loc='upper left')
axx.set(ylabel='nonthermal magnitude')

plt.show()

In [ ]:
def normalize(s):
    return np.nan_to_num(s / s.sum())

norm_th = normalize(thermal_basis)
norm_nth = normalize(nonthermal_basis)

fig, ax = plt.subplots()
ax.stairs(norm_th, t, label='thermal')
ax.stairs(norm_nth, t, label='nonthermal')
plt.show()

In [ ]:
thermal_prop = np.round((thermal_photons / total_photons).to_value(u.one), 3)
nonthermal_prop = np.round(1 - thermal_prop, 3)

spectrogram = list()

for i in range(energies.size - 1):
    ai = thermal_prop[i]
    bi = nonthermal_prop[i]
    decomp_part = (ai*norm_th + bi*norm_nth)
    spectrogram.append(
        total_photons[i].sum() * decomp_part
    )

spectrogram = np.array(spectrogram)

In [ ]:
tests = np.arange(spectrogram.shape[0])

fig, ax = plt.subplots()
for test in tests:
    ax.stairs(spectrogram[test], t)
ax.set(xlabel='time (s)', ylabel='photons incident')
plt.show()

In [ ]:
reconstructed = spectrogram.sum(axis=1)
fig, ax = plt.subplots()

with viz.quantity_support():
    ax.stairs(total_photons, energies, label='original data')
    ax.stairs(reconstructed, energies, label='reconstructed data')

ax.legend()
ax.set(xscale='log', yscale='log')
plt.show()